In [142]:
from sklearn.datasets import load_wine
from datetime import timedelta
import pandas as pd
import numpy as np

In [143]:
read_data_file = pd.read_pickle('../flink.pkl')
df = pd.DataFrame(read_data_file)

In [144]:
df.columns

Index(['commit_befor ', 'commit_after', 'cal_smell', 'dispen_bf', 'object_bf',
       'bloater_bf', 'dispen_at', 'object_at', 'bloater_af', 'open_time',
       'closed_time', 'cal_time', 'changed_file', 'additions', 'deletions',
       'dev', 'cal_time_binaly'],
      dtype='object')

In [145]:
df['cal_time'][0:20].median()

Timedelta('7 days 05:57:00')

In [146]:
df['cal_time_binaly'] = df['cal_time'].apply(lambda x:  0 if x < timedelta (days = 7) else 1)

In [147]:
#X = df[['commit_befor ', 'dispen_bf', 'object_bf', 'bloater_bf', 'open_time',]]
X = df[['dispen_bf', 'object_bf', 'bloater_bf',]]
y = df['cal_time_binaly']

In [148]:
X = X.dropna()
y = y.dropna()

In [149]:
X_train ,X_test ,y_train , y_test = X[:20], X[20:],y[:20],y[20:]

In [150]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
clf = make_pipeline(StandardScaler(), SVC(gamma='auto' , random_state = 2))

In [151]:
from sklearn.model_selection import cross_val_score
cross_val_score(clf , X , y , cv=5 , scoring= "accuracy")

array([0.        , 0.66666667, 0.5       , 0.5       , 0.5       ])

In [152]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(clf,X,y,cv=5)
y_pred

array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [153]:
confusion_matrix(y,y_pred)

array([[ 0, 12],
       [ 5, 13]], dtype=int64)

In [154]:
from sklearn.metrics import precision_score, recall_score, f1_score
print('precision = ', precision_score(y,y_pred))
print('recall_score = ', recall_score(y,y_pred))
print('f1_score',f1_score(y,y_pred))

precision =  0.52
recall_score =  0.7222222222222222
f1_score 0.6046511627906976


In [155]:
frame={"y":y,"y_pred": y_pred}
pd.DataFrame(frame).sum()

y         18
y_pred    25
dtype: int64

In [156]:
from sklearn.model_selection import GridSearchCV

parameters = {
  
    'C': [0.01 , 0.1 ,1],
    'gamma' : ['scale' , 'auto'],
    'random_state':[2]
  
}
svc = SVC()
clf = GridSearchCV(svc , parameters , cv=3 , scoring='f1_macro')
clf.fit(X,y)

GridSearchCV(cv=3, estimator=SVC(),
             param_grid={'C': [0.01, 0.1, 1], 'gamma': ['scale', 'auto'],
                         'random_state': [2]},
             scoring='f1_macro')

In [157]:
clf.predict(X_test)

array([1, 0, 1, 1, 0, 0, 1, 1, 0, 1], dtype=int64)

In [158]:
clf.best_estimator_

SVC(C=1, gamma='auto', random_state=2)

In [159]:
y_pred = clf.predict(X)
y_pred

array([1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 1, 0, 1], dtype=int64)

In [160]:
clf.predict(X_test)

array([1, 0, 1, 1, 0, 0, 1, 1, 0, 1], dtype=int64)

In [161]:
clf.best_estimator_

SVC(C=1, gamma='auto', random_state=2)

In [162]:
y_pred_clf = clf.best_estimator_.predict(X)
y_pred_clf

array([1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 1, 0, 1], dtype=int64)

In [163]:
from sklearn.metrics import precision_score, recall_score, f1_score
print('precision = ', precision_score(y,y_pred_clf ,average='macro'))
print('recall_score = ', recall_score(y,y_pred_clf ,average='macro'))
print('f1_score',f1_score(y,y_pred_clf ,average='macro'))

precision =  0.8333333333333333
recall_score =  0.8472222222222222
f1_score 0.8316498316498316


In [164]:
#frame={"y":y,"y_pred_clf": y_pred_clf}
#pd.DataFrame(frame)

In [165]:
from matplotlib import pyplot as plt
from sklearn import svm
import numpy as np

def f_importances(coef, names):
    
    names = np.arange(1)
    imp = coef
    imp,names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.show()

features_names = X.columns
svm1 = svm.SVC(kernel='linear',C=1)
svm1.fit(X, y)

SVC(C=1, kernel='linear')

In [166]:
svm1.coef_

array([[-9.98135817e-05,  5.55111512e-16,  3.30673909e-03]])

In [167]:
import itertools
from itertools import product 

In [173]:
result = itertools.combinations(X ,2)

for item in result:
    df  = pd.DataFrame(item)
    X[list (item)]
    parameters = {
    'C': [0.01 , 0.03 , 0.05 ],
    'gamma' : ['auto'],
    'random_state':[2]
    }
    svc = SVC()
    clf = GridSearchCV(svc , parameters , cv=3 , scoring='f1_macro')
    clf.fit(X_train[list (item)],y_train)
    clf.predict(X_test[list (item)])
    #print('predict=',clf.predict(X_test[list (item)]))
    #print(X_test)
    #print("y_train = ",y_train)
    print(item)
    print("precision_score =",precision_score(clf.predict(X[list(item)]),y))
    print("recall_score =", recall_score(clf.predict(X[list(item)]),y))
    print("f_1=",f1_score(clf.predict(X[list(item)]),y,average='macro'))
    #print("f_1_X-test=",f1_score(clf.predict(X_test[list(item)]),y_test,average='macro'))
    

('dispen_bf', 'object_bf')
precision_score = 0.7222222222222222
recall_score = 0.8666666666666667
f_1= 0.7643097643097643
('dispen_bf', 'bloater_bf')
precision_score = 0.3888888888888889
recall_score = 1.0
f_1= 0.6228571428571428
('object_bf', 'bloater_bf')
precision_score = 0.5
recall_score = 0.75
f_1= 0.6


In [169]:
itertools.combinations_with_replacement(X,2)

In [170]:
l = list(itertools.combinations(X,2))
print(l)

[('dispen_bf', 'object_bf'), ('dispen_bf', 'bloater_bf'), ('object_bf', 'bloater_bf')]


In [171]:
print([''.join(i) for i in itertools.combinations(pd.DataFrame(X), 2)])

['dispen_bfobject_bf', 'dispen_bfbloater_bf', 'object_bfbloater_bf']
